In [1]:
import asyncio
import random
from collections import deque

import mlflow
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from gymnasium.spaces import Box, Discrete
from loguru import logger
from poke_env import MaxBasePowerPlayer, Player, RandomPlayer, SimpleHeuristicsPlayer
from poke_env.battle import AbstractBattle
from poke_env.data import GenData

from params import MLFLOW_EXPERIMENT_NAME, MLFLOW_HOST
from registry import load_model, save_model


In [2]:
mlflow.set_tracking_uri(MLFLOW_HOST)
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

2025/09/08 16:40:55 INFO mlflow.tracking.fluent: Experiment with name 'PYC-AI-CHU' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/253089054422963212', creation_time=1757342456978, experiment_id='253089054422963212', last_update_time=1757342456978, lifecycle_stage='active', name='PYC-AI-CHU', tags={}>

In [3]:
class QNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ELU(),
            nn.Linear(128, 64),
            nn.ELU(),
            nn.Linear(64, output_dim),
        )
    def forward(self, x):
        return self.net(x)

In [4]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    def push(self, s, a, r, s2, done):
        self.buffer.append((s, a, r, s2, done))
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        s, a, r, s2, d = map(np.stack, zip(*batch))
        return (
            torch.tensor(s, dtype=torch.float32),
            torch.tensor(a, dtype=torch.long),
            torch.tensor(r, dtype=torch.float32),
            torch.tensor(s2, dtype=torch.float32),
            torch.tensor(d, dtype=torch.float32),
        )
    def __len__(self):
        return len(self.buffer)

In [ ]:
class SimpleRLAgent(Player):
    observation_space = Box(low=-1.0, high=4.0, shape=(10,), dtype=np.float32)

    def __init__(self, battle_format: str, q_net:QNetwork, buffer: ReplayBuffer, device=None,**kwargs):
        super().__init__(battle_format=battle_format, **kwargs)
        self.q_net = q_net
        self.buffer = buffer
        self.epsilon = 1.0
        self.device = device if device is not None else torch.device("cpu")

        # Store previous state and action for experience replay
        self.prev_state = None
        self.prev_action = None
        self.prev_battle_id = None

    def embed_battle(self, battle: AbstractBattle):
        moves_base_power = -np.ones(4, dtype=np.float32)
        moves_dmg_multiplier = np.ones(4, dtype=np.float32)
        type_chart = GenData.from_gen(9).type_chart  # For Gen 9 format

        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = move.base_power / 100
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1, battle.opponent_active_pokemon.type_2, type_chart=type_chart
                )

        fainted_team = len([mon for mon in battle.team.values() if mon.fainted]) / 6
        fainted_opponent = len([mon for mon in battle.opponent_team.values() if mon.fainted]) / 6

        return np.concatenate(
            [moves_base_power, moves_dmg_multiplier, [fainted_team, fainted_opponent]], axis=0
        ).astype(np.float32)

    def reward_computing_helper(
            self,
            battle: AbstractBattle,
            *,
            fainted_value: float = 0.15,
            hp_value: float = 0.15,
            number_of_pokemons: int = 6,
            starting_value: float = 0.0,
            status_value: float = 0.15,
            victory_value: float = 1.0
    ) -> float:
        """Compute reward based on battle state with configurable parameters

        Args:
            battle: The current battle state
            fainted_value: Weight for fainted pokemon difference
            hp_value: Weight for HP difference
            number_of_pokemons: Total number of pokemon per team
            starting_value: Base reward value
            status_value: Weight for status condition difference
            victory_value: Bonus for winning the battle
        """
        if battle.finished:
            return victory_value if battle.won else -victory_value

        # Initialize reward with starting value
        reward = starting_value

        # 1. Fainted pokemon component
        fainted_team = len([mon for mon in battle.team.values() if mon.fainted])
        fainted_opponent = len([mon for mon in battle.opponent_team.values() if mon.fainted])

        # Normalize by number of pokemon and apply weight
        fainted_diff = (fainted_opponent - fainted_team) / number_of_pokemons
        reward += fainted_value * fainted_diff

        # 2. HP component
        team_hp_fraction = 0.0
        team_alive = 0
        for mon in battle.team.values():
            if not mon.fainted and mon.current_hp_fraction is not None:
                team_hp_fraction += mon.current_hp_fraction
                team_alive += 1

        opponent_hp_fraction = 0.0
        opponent_alive = 0
        for mon in battle.opponent_team.values():
            if not mon.fainted and mon.current_hp_fraction is not None:
                opponent_hp_fraction += mon.current_hp_fraction
                opponent_alive += 1

        # Normalize HP by number of pokemon and apply weight
        if team_alive > 0 and opponent_alive > 0:
            team_avg_hp = team_hp_fraction / number_of_pokemons
            opponent_avg_hp = opponent_hp_fraction / number_of_pokemons
            hp_diff = opponent_avg_hp - team_avg_hp  # Positive if opponent has more HP
            reward += hp_value * hp_diff

        # 3. Status conditions component
        team_status_count = 0
        opponent_status_count = 0

        for mon in battle.team.values():
            if not mon.fainted and mon.status is not None:
                team_status_count += 1

        for mon in battle.opponent_team.values():
            if not mon.fainted and mon.status is not None:
                opponent_status_count += 1

        # Normalize status count and apply weight (negative status is bad for us)
        status_diff = (team_status_count - opponent_status_count) / number_of_pokemons
        reward -= status_value * status_diff  # Subtract because status is negative

        return reward

    def compute_reward(self, battle: AbstractBattle) -> float:
        return self.reward_computing_helper(battle, fainted_value=2, hp_value=1, victory_value=15)

    def choose_move(self, battle: AbstractBattle):
        current_state = self.embed_battle(battle)

        # Store experience from previous step
        if self.prev_state is not None and self.prev_action is not None and self.prev_battle_id == battle.battle_tag:
            reward = self.compute_reward(battle)
            done = battle.finished

            self.buffer.push(self.prev_state, self.prev_action, reward, current_state, done)

        # Choose action
        obs = torch.tensor(current_state, dtype=torch.float32, device=self.device)
        if random.random() < self.epsilon:
            action = self._get_random_action(battle)
        else:
            with torch.no_grad():
                q = self.q_net(obs.unsqueeze(0))
            action = q.argmax(dim=1).item()

        # Store current state and action for next step
        self.prev_state = current_state
        self.prev_action = action
        self.prev_battle_id = battle.battle_tag


        return self._action_to_move(action, battle)

    def choose_switch(self, battle: AbstractBattle):
        return self.choose_move(battle)

    def _get_random_action(self, battle: AbstractBattle):
        """Get a random valid action index"""
        total_options = len(battle.available_moves) + len(battle.available_switches)
        return random.randint(0, total_options)

    def _action_to_move(self, action, battle: AbstractBattle):
        options = battle.available_moves + battle.available_switches
        if len(options) == 0:
            # Fallback to random move if no options available
            return self.choose_random_move(battle)
        move = options[action % len(options)]
        return self.create_order(move)

    async def _battle_finished_callback(self, battle: AbstractBattle):
        """Called when a battle finishes - store final experience"""
        if self.prev_state is not None and self.prev_action is not None and self.prev_battle_id == battle.battle_tag:
            final_reward = self.compute_reward(battle)
            final_state = self.embed_battle(battle)

            self.buffer.push(
                self.prev_state,
                self.prev_action,
                final_reward,
                final_state,
                True,  # Battle is finished
            )

        # Reset for next battle
        self.prev_state = None
        self.prev_action = None
        self.prev_battle_id = None

        await super()._battle_finished_callback(battle)

In [6]:
def train_epoch(agent, target_net, buffer, optimizer,
                batch_size: int = 4, gamma: float = 0.99,
                eps_start: float = 1.0, eps_end: float = 0.05, eps_decay: float = 1e-4,
                update_target_every: int = 1000):
    mlflow.log_params({
        "gamma": 0.99,
        "batch_size": 4,
        "lr": 1e-3
        })
    # Decrease epsilon (linear decay)
    agent.epsilon = max(eps_end, agent.epsilon - eps_decay)

    # Skip training if buffer is too small
    if len(buffer) < batch_size:
        return

    # Sample a batch of transitions
    s, a, r, s2, done = buffer.sample(batch_size)
    # Predictions for current states
    q_values = agent.q_net(s).gather(1, a.unsqueeze(1)).squeeze(1)

    # Compute target values using target network
    with torch.no_grad():
        max_next_q = target_net(s2).max(1)[0]
        targets = r + gamma * (1 - done) * max_next_q

    # Compute loss and update network
    loss = F.mse_loss(q_values, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Periodically sync target network
    train_step = getattr(agent, "train_step_count", 0) + 1
    agent.train_step_count = train_step
    if train_step % update_target_every == 0:
        target_net.load_state_dict(agent.q_net.state_dict())


In [7]:
input_dim = 10
output_dim = 26  # discrete number of possible moves
q_net = QNetwork(input_dim, output_dim)
q_net = q_net.cuda()
target_net = QNetwork(input_dim, output_dim)
target_net.load_state_dict(q_net.state_dict())
target_net = target_net.cuda()
optimizer = optim.Adam(q_net.parameters(), lr=1e-3)
buffer = ReplayBuffer(capacity=1000)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

agent = SimpleRLAgent(battle_format="gen9randombattle", q_net=q_net, buffer=buffer, device=device)
opponent = MaxBasePowerPlayer(battle_format="gen9randombattle")

/home/bapti/code/blegeron/pokemon/poke-agent/.venv/lib/python3.12/site-packages/poke_env/player/player.py:323: RuntimeWarning: coroutine 'SimpleRLAgent._battle_finished_callback' was never awaited
  self._battle_finished_callback(battle)
/home/bapti/code/blegeron/pokemon/poke-agent/.venv/lib/python3.12/site-packages/poke_env/player/player.py:323: RuntimeWarning: coroutine 'SimpleRLAgent._battle_finished_callback' was never awaited
  self._battle_finished_callback(battle)
/home/bapti/code/blegeron/pokemon/poke-agent/.venv/lib/python3.12/site-packages/poke_env/player/player.py:323: RuntimeWarning: coroutine 'SimpleRLAgent._battle_finished_callback' was never awaited
  self._battle_finished_callback(battle)
/home/bapti/code/blegeron/pokemon/poke-agent/.venv/lib/python3.12/site-packages/poke_env/player/player.py:323: RuntimeWarning: coroutine 'SimpleRLAgent._battle_finished_callback' was never awaited
  self._battle_finished_callback(battle)
/home/bapti/code/blegeron/pokemon/poke-agent/.ve

In [9]:
device = next(agent.q_net.parameters()).device

In [10]:
def train_step(agent, target_net, buffer, optimizer, batch_size=4, gamma=0.99):
    if len(buffer) < batch_size:
        return

    s, a, r, s2, done = buffer.sample(batch_size)

    # Déplace tous les tensors du batch
    s = s.to(device)
    a = a.to(device)
    r = r.to(device)
    s2 = s2.to(device)
    done = done.to(device)

    q_values = agent.q_net(s).gather(1, a.unsqueeze(1)).squeeze(1)

    with torch.no_grad():
        max_next_q = target_net(s2).max(1)[0]
        targets = r + gamma * (1 - done) * max_next_q

    loss = F.mse_loss(q_values, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

In [11]:
async def train_agent():
    for epoch in range(10):  # 10 epochs
        logger.info(f"Epoch {epoch + 1} started.")
        epoch_losses = []

        for battle_num in range(100):  # 100 battles per epoch
            logger.info(f"  Battle {battle_num + 1} started.")
            await agent.battle_against(opponent, n_battles=1)
            # Perform training after each battle
            loss = train_step(agent, target_net, buffer, optimizer)
            if loss is not None:
                epoch_losses.append(loss)

            logger.info(f"  Battle {battle_num + 1} finished.")

        logger.info(
            {
                "lose_rate": agent.n_lost_battles / agent.n_finished_battles,
                "win_rate": agent.n_won_battles / agent.n_finished_battles,
                "avg_loss": np.mean(epoch_losses) if epoch_losses else 0,
            }
        )

        if epoch % 2 == 0:
            mlflow.pytorch.log_model(
                pytorch_model=agent.q_net,
                name="dqn_model",
                input_example=np.zeros((1, 1, 10), dtype=np.float32),
                step=epoch,
            )

        logger.info(f"Epoch {epoch + 1} finished.")

In [12]:
await train_agent()

2025-09-08 16:40:57.108 | INFO     | __main__:train_agent:3 - Epoch 1 started.
2025-09-08 16:40:57.109 | INFO     | __main__:train_agent:7 -   Battle 1 started.
2025-09-08 16:40:57.773 | INFO     | __main__:train_agent:14 -   Battle 1 finished.
2025-09-08 16:40:57.774 | INFO     | __main__:train_agent:7 -   Battle 2 started.
2025-09-08 16:40:57.881 | INFO     | __main__:train_agent:14 -   Battle 2 finished.
2025-09-08 16:40:57.881 | INFO     | __main__:train_agent:7 -   Battle 3 started.
2025-09-08 16:40:58.077 | INFO     | __main__:train_agent:14 -   Battle 3 finished.
2025-09-08 16:40:58.078 | INFO     | __main__:train_agent:7 -   Battle 4 started.
2025-09-08 16:40:58.180 | INFO     | __main__:train_agent:14 -   Battle 4 finished.
2025-09-08 16:40:58.181 | INFO     | __main__:train_agent:7 -   Battle 5 started.
2025-09-08 16:40:58.289 | INFO     | __main__:train_agent:14 -   Battle 5 finished.
2025-09-08 16:40:58.290 | INFO     | __main__:train_agent:7 -   Battle 6 started.
2025-09-0

In [13]:
# n_won_battles and n_finished_battles
mlflow.log_metrics({'win_ratio':agent.n_won_battles/agent.n_finished_battles})
logger.debug(
    f"Player {agent.username} won {agent.n_won_battles} out of {\
        agent.n_finished_battles} played"
)

mlflow.pytorch.log_model(
        pytorch_model=agent.q_net,
        name="dqn_model",
        input_example=np.zeros((1, 1, 10), dtype=np.float32)
    )


2025-09-08 16:43:39.384 | DEBUG    | __main__:<module>:3 - Player SimpleRLAgent 1 won 103 out of 1000 played
2025/09/08 16:43:40 WARNING mlflow.models.signature: Failed to infer the model signature from the input example. Reason: RuntimeError('Expected all tensors to be on the same device, but got mat1 is on cpu, different from other tensors on cuda:0 (when checking argument in method wrapper_CUDA_addmm)'). To see the full traceback, set the logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)`.
2025/09/08 16:43:42 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/09/08 16:43:42 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": [
    [
      [
        0.0,
       .... Alternatively, you can avoid passing input example and pass model signature instead when logging the model. To ensure the input example is valid prior

In [14]:
save_model(agent.q_net, 'pokemon_torch_rl')

In [21]:
agent_test = SimpleRLAgent(battle_format="gen9randombattle", q_net=load_model('pokemon_torch_rl'), buffer=buffer)

/home/bapti/code/blegeron/pokemon/poke-agent/src/models/pokemon_torch_rl.pkl
<_io.BufferedReader name='/home/bapti/code/blegeron/pokemon/poke-agent/src/models/pokemon_torch_rl.pkl'>


In [20]:
await agent_test.send_challenges("blegeron", n_challenges=1)